In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from loguru import logger

import pickle
import requests
import json
import re
from pathlib import Path

data_dir = Path(globals()['_dh'][0]).parent / 'data'
figure_dir = data_dir / 'figures' / 'get_transfer_costs'
figure_dir.mkdir(exist_ok=True)

plt.style.use('seaborn-bright')
plt.set_cmap('plasma')

<Figure size 432x288 with 0 Axes>

In [3]:
df = pd.read_json('../data/out.csv')
df

ValueError: Expected object or value